In [ ]:
from helper import *
from postProcessHelper import *

config = Config()
config.setFromEnv()

versionControl = 'V3'
promptVersion='P2'
saveName = f"{versionControl}-{promptVersion}"

config.setSaveDetails(saveName)

chartFolder = os.path.join(config.baseOutputFolder, \
                           config.outputFolders['CHART_OUTPUT'], \
                           config.fullName)

excelFolder = os.path.join(config.baseOutputFolder, \
                           config.outputFolders['EXCEL_OUTPUT'], \
                           config.fullName)

In [ ]:
resultsDF = convertPicklesToDF('saves', config)
errorDF = convertPicklesToDF('errors', config)

mergedCriterionData = getCriterionDataDF(resultsDF, saveName, excelFolder)

In [ ]:
getScoreSpread(resultsDF, chartFolder)

## Generate Histogram spread of scores by Assignment

In [ ]:
getFullHistogramSpread(resultsDF, chartFolder)

## Generate Scatterplot spread of scores by Assignment

In [ ]:
getFullScatterplotSpread(resultsDF, chartFolder)

## Generate Histogram spread of scores by Criterion per Assignment

In [ ]:
getHistogramSpreadByAssgn(resultsDF, chartFolder)

## Generate Scatterplot spread of scores by Criterion per Assignment

In [ ]:
getScatterplotSpreadByAssgn(resultsDF, chartFolder)

## Track Difference in scores per Assignment

In [ ]:
saveGraderPeerGPTMeanScoreDiff(resultsDF, saveName, excelFolder)

## Track difference in score per Criteria

In [ ]:
fullInfoDF, rubricOrderDict = buildFullInfoDF(config, resultsDF, saveName, excelFolder)

In [ ]:
getMeanDiffCharts(fullInfoDF, rubricOrderDict, chartFolder)

In [ ]:
getMeanDiffPercentCharts(fullInfoDF, rubricOrderDict, chartFolder)

In [ ]:
meanConDF, retrievedCIDF = getCIOutlierGraderDetails(fullInfoDF, saveName, excelFolder, confidenceLevel=0.93)

display(retrievedCIDF)

In [ ]:
getHighErrorCriteria(config, fullInfoDF, saveName, excelFolder, scoreThreshold = 10)

## Misc. Code Snippets

In [ ]:
from helper import *

config = Config()
config.setFromEnv()
config.customDescMode = True

versionControl = 'V3'
promptVersion='P2'
fullName = f'{versionControl}-{promptVersion}'
config.setSaveDetails(fullName)

saveName = f"{versionControl}-{promptVersion}"


gradeRubricAssignmentDF = getGRAData(config)

for index, row in gradeRubricAssignmentDF.iterrows():
    fullCriterionDF = getRowCriterionDF(row, config)
    fullCritText = buildCritPrompt(fullCriterionDF, useCustomDesc=config.customDescMode)
    studentSubmission = getSubmissionText(row['assignment_id'], row['submitter_id'], config)

    if studentSubmission:
        promptVariableDict = {
                            'Course Name': config.simpleCourseName,
                            'Assignment Name': row['assignment_title'],
                            'Assignment Description': row['cleaned_description'],
                            'Student Submission': studentSubmission,
                            'Criterion Description and Rubric': fullCritText,
                            'Maximum Points': row['points_possible'],
                            }
        fullPrompt = promptBuilder(promptVariableDict, useCustomDesc=config.customDescMode)

        # print(fullPrompt)

        with open(os.path.join(config.baseOutputFolder, \
                               config.outputFolders['PROMPT_FILES'], \
                               config.fullName, \
                               f'{config.fullName}_exampleFilledPrompt.txt'), 'w') as textFile:
            textFile.write(fullPrompt)
        break

In [ ]:
errorDF